# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("weather_data.csv")
df.head()

,city,Country,Latitude,Longitude,Temperature,Date,wind_Speed,Pressure,Humidity,Cloudiness
0,Kapaa,US,22.0752,-159.3190,76.89,1644441062,5.99,1018.0,68.0,25.0
1,Rikitea,PF,-23.1203,-134.9692,79.00,1644441062,15.26,1020.0,68.0,0.0
2,Cape Town,ZA,-33.9258,18.4232,68.54,1644441062,1.14,1014.0,79.0,0.0
3,Hilo,US,19.7297,-155.0900,74.79,1644441062,3.44,1019.0,72.0,100.0
4,Nushki,PK,29.5542,66.0215,48.31,1644441062,2.19,1021.0,29.0,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         609 non-null    object 
 1   Country      549 non-null    object 
 2   Latitude     556 non-null    float64
 3   Longitude    556 non-null    float64
 4   Temperature  556 non-null    float64
 5   Date         609 non-null    int64  
 6   wind_Speed   556 non-null    float64
 7   Pressure     556 non-null    float64
 8   Humidity     556 non-null    float64
 9   Cloudiness   556 non-null    float64
dtypes: float64(7), int64(1), object(2)
memory usage: 47.7+ KB


In [6]:
df2 =df.dropna().reset_index(drop=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549 entries, 0 to 548
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         549 non-null    object 
 1   Country      549 non-null    object 
 2   Latitude     549 non-null    float64
 3   Longitude    549 non-null    float64
 4   Temperature  549 non-null    float64
 5   Date         549 non-null    int64  
 6   wind_Speed   549 non-null    float64
 7   Pressure     549 non-null    float64
 8   Humidity     549 non-null    float64
 9   Cloudiness   549 non-null    float64
dtypes: float64(7), int64(1), object(2)
memory usage: 43.0+ KB


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
 # Access maps with API key
gmaps.configure(api_key=g_key)

In [9]:
locations = df2.loc[:, ["Latitude", "Longitude"]]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df2['Humidity'],
    max_intensity=20, point_radius=5
)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [39]:
mask = (df2.Temperature < 80) & (df2.wind_Speed < 10) & (df2.Temperature > 70) & (df2.Cloudiness == 0)

df3 = df2.loc[mask]
df3

,city,Country,Latitude,Longitude,Temperature,Date,wind_Speed,Pressure,Humidity,Cloudiness
144,Burgersdorp,ZA,-30.9977,26.3286,72.48,1644441062,7.54,1010.0,27.0,0.0
333,Pacific Grove,US,36.6177,-121.9166,75.20,1644441062,5.75,1020.0,30.0,0.0
370,Bonthe,SL,7.5264,-12.5050,78.48,1644441062,7.16,1010.0,85.0,0.0
396,Yulara,AU,-25.2406,130.9889,75.04,1644441062,9.22,1014.0,50.0,0.0
458,Springbok,ZA,-29.6643,17.8865,71.06,1644441062,4.14,1013.0,32.0,0.0
514,Yeppoon,AU,-23.1333,150.7333,71.38,1644441062,6.20,1012.0,70.0,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
hotel_name = []
ratings = []
addresses = []
cities = []
countries = []
lats = []
longs = []

for _,row in df3.iterrows():
    print(row.city)

    # coordinates
    target_coordinates = f"{row.Latitude}, {row.Longitude}"
    target_search = "hotel"

    target_radius = 5000
    target_type = "lodging"

    # parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    places_data = response.json()

    hotel = places_data["results"][0]["name"]
    address = places_data["results"][0]["vicinity"]
    rating = places_data["results"][0]["rating"]

    # put it all in a list
    hotel_name.append(hotel)
    ratings.append(rating)
    addresses.append(address)
    cities.append(row.city)
    countries.append(row.Country)
    lats.append(row.Latitude)
    longs.append(row.Longitude)

Burgersdorp
Pacific Grove
Bonthe
Yulara
Springbok
Yeppoon


In [43]:
hotel_df = pd.DataFrame()

hotel_df['City'] = cities
hotel_df['Country'] = countries
hotel_df['Latitude'] = lats
hotel_df['Longitude'] = longs
hotel_df['Hotel'] = hotel_name
hotel_df['Address'] = addresses
hotel_df['Rating'] = ratings

hotel_df

,City,Country,Latitude,Longitude,Hotel,Address,Rating
0,Burgersdorp,ZA,-30.9977,26.3286,The Kingsman boutique hotel,"10 Piet Retief St, Burgersdorp",4.4
1,Pacific Grove,US,36.6177,-121.9166,Hotel 1110,"1110 Del Monte Ave, Monterey",4.0
2,Bonthe,SL,7.5264,-12.5050,Bonthe Holiday Village,"GFGX+7Q3, Bonthe",4.5
3,Yulara,AU,-25.2406,130.9889,The Lost Camel Hotel - Ayers Rock Resort,"Ayers Rock, Yulara Dr, Yulara",4.0
4,Springbok,ZA,-29.6643,17.8865,Springbok Inn by Country Hotels,"Voortrekker St, Springbok",4.0
5,Yeppoon,AU,-23.1333,150.7333,Coral Inn Boutique Hotel,"14 Maple St, Yeppoon",3.9


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [1]:
# Add marker layer ontop of heat map
# create layers
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(
    locations, weights=df3['Humidity'],
    max_intensity=20, point_radius=5
)


fig = gmaps.figure()

# add layers
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)

# Display figure
fig

NameError: name 'gmaps' is not defined